In [1]:
import os
import gym
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import Adam
from collections import deque

In [2]:
env = gym.make("MountainCar-v0")

In [3]:
class mountain_agent: 
    
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.gamma = 0.95
        self.lr_rate = 0.001
        self.model = self.build_model()
        
    def build_model(self):
        model = Sequential()
        
        model.add(Dense(24,activation='relu',input_dim=self.state_size))
        model.add(Dense(24,activation='relu'))
        model.add(Dense(self.action_size,activation='linear'))
        
        adam = Adam(lr=self.lr_rate)
        model.compile(loss='mse',optimizer=adam)
        
        return model
    
    def rem(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0])
    
    def train(self,batch_size=32):
        minibatch = random.sample(self.memory,batch_size)
        
        for state,action,reward,next_state,done in minibatch:
            if not done:
                target = reward + self.gamma*np.argmax(self.model.predict(next_state)[0])
            else:
                target = reward
                
            target_f = self.model.predict(state)
            target_f[0][action] = target
            
            self.model.fit(state,target_f,epochs=1,verbose=0)
            
        if self.epsilon_min < self.epsilon:
            self.epsilon *= self.epsilon_decay

### Train Model

In [4]:
state_size = 2
action_size = 3
done=False
ag = mountain_agent(state_size=state_size,action_size=action_size)

In [5]:
for e in range(500):
    state=env.reset()
    state=np.reshape(state,[1,state_size])
    batch_size=32
    
    for time in range(100):
        env.render()
        action=ag.act(state)
        next_state,reward,done,other_info=env.step(action)
        
        if next_state[1] > state[0][1] and next_state[1]>0 and state[0][1]>0:
            reward += 15
        elif next_state[1] < state[0][1] and next_state[1]<=0 and state[0][1]<=0:
            reward +=15
                
        if done:
            reward = reward + 10000
        else:
            # put a penalty if the no of time steps is more
            reward = reward - 10
            
        next_state=np.reshape(next_state,[1,state_size])
        ag.rem(state,action,reward,next_state,done)
        state=next_state
        
        if done:
            print(f'Game episode: {e+1}/{1000}, High Score: {time}, Exploration rate: {ag.epsilon}')
            break
        
    if len(ag.memory)>batch_size:
        ag.train(batch_size)

    
print('Deep Q learner Trained')
env.close()      

KeyboardInterrupt: 